# Код Хэмминга #
В данной работе будет рассмотрен кодирование и декодирование сообщения, нахождение и справление одиночной ошибки.

In [15]:
import numpy as np

n = 7
r = int(np.log2(n+1)) if int(np.log2(n+1))==float(np.log2(n+1)) else int(np.log2(n+1))+1
k = n-r

## Построение проверочной матрицы ## 
Столбцы проверочной матрицы состоят из всех ненулевых бинарных векторов длины r, каждый вектор встречается ровно один раз.

In [16]:
syndrome_matrix = np.array([list(map(int, format(xi, "0"+str(r)+"b"))) for xi in range(1, n+1)]).transpose()
syndrome_matrix

array([[0, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 1]])

## Построение порождающей матрицы ## 
Для построения порождающей матрицы выберем подматрицу *syndrome_matrix*, состоящую из столбцов с номерами не из степеней 2. 
Первая строка полушившейся матрицы <em>A</em> отвечает за последний проверочной бит, вторая за предпоследний и т. д. Исходя из 
вышесказанного, вставим строки матрицы <em>A</em> в качестве столбцов матрицы *syndrome_matrix* на позиции: последнюю троку на 
позицию 2^0, предпоследнюю строку на позицию 2^1 и т. д. Остальные столбцы будут столбцами единочной матрицы.

In [17]:
generator_matrix = np.zeros([k, n])
A = syndrome_matrix[:, [x-1 for x in range(1, n+1) if int(np.log2(x)) != float(np.log2(x))]]
A = A.transpose()
A = np.flip(A, axis=1)
for i in range(0, n):
    if int(np.log2(i+1)) == float(np.log2(i+1)):
        generator_matrix[:, i] = A[:, int(np.log2(i+1))]
    else:
        generator_matrix[i-len(format(i+1, "b")), i] = 1
generator_matrix

array([[1., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0.],
       [1., 1., 0., 1., 0., 0., 1.]])

## Кодирование сообщения ## 

In [18]:
hamming_message = np.array([1, 0, 1, 0]).dot(generator_matrix)
hamming_message = np.array(list(map(lambda x: x%2, hamming_message)))
hamming_message

array([1., 0., 1., 1., 0., 1., 0.])

## Декодирование сообщения ## 

In [19]:
error_message = np.array([1, 1, 1, 1, 0, 1, 0])
syndrome = syndrome_matrix.dot(error_message.transpose())
syndrome = list(map(lambda x: x%2, syndrome))
syndrome

[0, 1, 0]